In [ ]:
import rospy
import ipywidgets as widgets
from IPython.display import display
from geometry_msgs.msg import PoseStamped 
import actionlib.msg 
import assignment_2_2022.msg 
from nav_msgs.msg import Odometry
from rt1_2nd_assignment.msg import odom_custom_msg
import os



def send_target(x, y):
    rospy.init_node('NodeA')
    client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)  # Replace 'your_action_server' and YourAction with your specific values
    client.wait_for_server()

    x_position = int(x)
    y_position = int(y)
    goal = PoseStamped()


    goal.pose.position.x = x_position
    goal.pose.position.y = y_position


    goal = assignment_2_2022.msg.PlanningGoal(goal)    
    client.send_goal(goal)
    client.wait_for_result()

def set_target_coordinates(button):
    x = float(x_text.value)
    y = float(y_text.value)
    send_target(x, y)

# Create the button and input fields
x_text = widgets.Text(description='X:')
y_text = widgets.Text(description='Y:')
button = widgets.Button(description='Set Target')

# Attach the button callback function
button.on_click(set_target_coordinates)

# Display the widgets
display(x_text, y_text, button)

In [ ]:
import matplotlib.pyplot as plt
import rospy
from nav_msgs.msg import Odometry
from matplotlib.animation import FuncAnimation
plt.switch_backend('TKAgg')

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []

    def plot_init(self): 
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln 

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y) 
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame): 
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

if __name__ == '__main__':
    if not rospy.core.is_initialized():
        rospy.init_node('odom_visualizer_node')
    vis = Visualiser()
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, save_count=100) 
    plt.show(block=True)

In [ ]:
import rospy
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets
from IPython.display import display

class LaserScannerVisualizer:
    def __init__(self):
        self.distance_text = widgets.FloatText(description='Closest Distance:', value=0.0)

    def laser_callback(self, msg):
        closest_distance = min(msg.ranges)
        self.distance_text.value = closest_distance

rospy.init_node('laser_visualizer_node')
vis = LaserScannerVisualizer()
sub = rospy.Subscriber('/scan', LaserScan, vis.laser_callback)

# Display the distance text box
# true
display(vis.distance_text)

In [ ]:
import rospy
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
plt.switch_backend('TKAgg')

class LaserScannerVisualizer:
    def __init__(self):
        self.distance_text = widgets.FloatText(description='Closest Distance:', value=0.0)
        self.reached_targets = 0
        self.not_reached_targets = 0
        self.reached_targets_list = []
        self.not_reached_targets_list = []

    def laser_callback(self, msg):
        closest_distance = min(msg.ranges)
        self.distance_text.value = closest_distance

        # Simulated condition to check if target is reached or not
        target_reached = (closest_distance < 1.0)  # Modify this condition as per your requirements

        if target_reached:
            self.reached_targets += 1
        else:
            self.not_reached_targets += 1

        self.reached_targets_list.append(self.reached_targets)
        self.not_reached_targets_list.append(self.not_reached_targets)

    def plot_statistics(self):
        plt.figure()
        plt.plot(self.reached_targets_list, label='Reached Targets')
        plt.plot(self.not_reached_targets_list, label='Not Reached Targets')
        plt.xlabel('Time')
        plt.ylabel('Number of Targets')
        plt.title('Reached/Not Reached Targets')
        plt.legend()
        plt.show()

# rospy.init_node('laser_visualizer_node')
vis = LaserScannerVisualizer()
sub = rospy.Subscriber('/scan', LaserScan, vis.laser_callback)

# Display the distance text box
display(vis.distance_text)

# Wait for some time to collect data
rospy.sleep(10.0)  # Adjust the sleep duration as per your requirements

# Plot the statistics
vis.plot_statistics()